# nb-model_xg-model-fr-model

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from collections import OrderedDict
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-model-fr-model.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset as TorchDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import torchfunc
from torchmeta.utils.data import BatchMetaDataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, pd_to_np, pairwise, df_midx_restack, compose, is_type, df_rows_in_year, list_all_eq, remove_dups_list, NestedDefaultDict, set_loglevel, search_df, chained_filter, get_variants, load_df, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import pd_rows_key_in, df_add_midx_level, df_del_midx_level, pd_split_ternary_to_binary, np_value_counts, isnt, window_iter, all_eq, np_assert_identical_len_dim, pd_idx_rename, midx_get_level, pd_rows, midx_intersect, pd_get_midx_level, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_at_least_nd, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.common import PYTORCH_ACT_MAPPING, PYTORCH_OPT_MAPPING, PYTORCH_SCH_MAPPING, PYTORCH_LOSS_MAPPING
from model.xg_util import xgload, get_xg_feature_dfs, get_xg_label_target_dfs
from model.preproc_util import temporal_preproc_3d, stride_preproc_3d
from model.train_util import pd_to_np_tvt, batchify
from model.pl_generic import GenericModel
from model.model_util import OutputLinear
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg-model-fr-model.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


Prune the xg data down to the data of interest to use in further experiments.

## Load Data

In [2]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

In [3]:
fd = get_xg_feature_dfs(chosen_asset)
ld, td = get_xg_label_target_dfs(chosen_asset)

INFO:root:pba_hlh_hlogret_hdzn_hdgau(8)_hret_hspread
INFO:root:pba_hoc_hlogret_hdzn_hdgau(8)_hret_hspread
INFO:root:pba_hohlca_hdzn_hdgau(8)
INFO:root:pba_hlh_hlogret_hdmx_hret_hspread
INFO:root:pba_hoc_hlogret_hdmx_hret_hspread
INFO:root:pba_hohlca_hdmx
INFO:root:pba_hlh_hlogret_hdod_hret_hspread
INFO:root:pba_hoc_hlogret_hdod_hret_hspread
INFO:root:pba_hohlca_hdod
INFO:root:pba_hlh_hlogret_hdpt_hret_hspread
INFO:root:pba_hoc_hlogret_hdpt_hret_hspread
INFO:root:pba_hohlca_hdpt
INFO:root:pba_hlh_hlogret_hdmx_hduni(8)_hret_hspread
INFO:root:pba_hoc_hlogret_hdmx_hduni(8)_hret_hspread
INFO:root:pba_hohlca_hdmx_hduni(8)
INFO:root:pba_hlh_hlogret_hdzn_hret_hspread
INFO:root:pba_hoc_hlogret_hdzn_hret_hspread
INFO:root:pba_hohlca_hdzn
INFO:root:pba_hohlca
INFO:root:vol_hlh_hlogret_hdzn_hdgau(8)_hret_hspread
INFO:root:vol_hoc_hlogret_hdzn_hdgau(8)_hret_hspread
INFO:root:vol_hohlca_hdzn_hdgau(8)
INFO:root:vol_hlh_hlogret_hdmx_hret_hspread
INFO:root:vol_hoc_hlogret_hdmx_hret_hspread
INFO:root:vo

INFO:root:pba_hlh_hdxcret1(0.062500%)_dxfbdir1_hdxcret1(0.125000%)_hdxcret1(0.250000%)_hdxcret1(0.500000%)_hdxcret1(1.000000%)_hdxcret1(2.000000%)_hdxcret1(4.000000%)_hdxcret1(8.000000%)
INFO:root:pba_hoc_hdxcret1(0.062500%)_dxfbdir1_hdxcret1(0.125000%)_hdxcret1(0.250000%)_hdxcret1(0.500000%)_hdxcret1(1.000000%)_hdxcret1(2.000000%)_hdxcret1(4.000000%)_hdxcret1(8.000000%)
INFO:root:pba_hlh_hdxcret2(0.5*avg,1)_dxfbdir2_hdxcret2(1*avg,1)_hdxcret2(2*avg,1)
INFO:root:pba_hoc_hdxcret2(0.5*avg,1)_dxfbdir2_hdxcret2(1*avg,1)_hdxcret2(2*avg,1)
INFO:root:pba_hlh_hdxcret2(0.5*mad,1)_dxfbdir2_hdxcret2(1*mad,1)_hdxcret2(2*mad,1)
INFO:root:pba_hoc_hdxcret2(0.5*mad,1)_dxfbdir2_hdxcret2(1*mad,1)_hdxcret2(2*mad,1)
INFO:root:pba_hlh_hdxcret2(0.5*max,1)_dxfbdir2_hdxcret2(1*max,1)_hdxcret2(2*max,1)
INFO:root:pba_hoc_hdxcret2(0.5*max,1)_dxfbdir2_hdxcret2(1*max,1)_hdxcret2(2*max,1)
INFO:root:pba_hlh_hdxcret2(0.5*min,1)_dxfbdir2_hdxcret2(1*min,1)_hdxcret2(2*min,1)
INFO:root:pba_hoc_hdxcret2(0.5*min,1)_dxfbdir

INFO:root:vol_hlh_hdxcret1(0.062500%)_dxfbdir1_hdxcret1(0.125000%)_hdxcret1(0.250000%)_hdxcret1(0.500000%)_hdxcret1(1.000000%)_hdxcret1(2.000000%)_hdxcret1(4.000000%)_hdxcret1(8.000000%)
INFO:root:vol_hoc_hdxcret1(0.062500%)_dxfbdir1_hdxcret1(0.125000%)_hdxcret1(0.250000%)_hdxcret1(0.500000%)_hdxcret1(1.000000%)_hdxcret1(2.000000%)_hdxcret1(4.000000%)_hdxcret1(8.000000%)
INFO:root:vol_hlh_hdxcret2(0.5*avg,1)_dxfbdir2_hdxcret2(1*avg,1)_hdxcret2(2*avg,1)
INFO:root:vol_hoc_hdxcret2(0.5*avg,1)_dxfbdir2_hdxcret2(1*avg,1)_hdxcret2(2*avg,1)
INFO:root:vol_hlh_hdxcret2(0.5*mad,1)_dxfbdir2_hdxcret2(1*mad,1)_hdxcret2(2*mad,1)
INFO:root:vol_hoc_hdxcret2(0.5*mad,1)_dxfbdir2_hdxcret2(1*mad,1)_hdxcret2(2*mad,1)
INFO:root:vol_hlh_hdxcret2(0.5*max,1)_dxfbdir2_hdxcret2(1*max,1)_hdxcret2(2*max,1)
INFO:root:vol_hoc_hdxcret2(0.5*max,1)_dxfbdir2_hdxcret2(1*max,1)_hdxcret2(2*max,1)
INFO:root:vol_hlh_hdxcret2(0.5*min,1)_dxfbdir2_hdxcret2(1*min,1)_hdxcret2(2*min,1)
INFO:root:vol_hoc_hdxcret2(0.5*min,1)_dxfbdir

In [4]:
d_all_common = ['dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd']
d_extra = ['dlogret', 'dohlca', 'ddiff', 'dc', 'dwrxmx']

In [5]:
h_all_common = ['hdmx', 'hdod', 'hdpt', 'hdzn']
h_extra = ['hdgau', 'hduni', 'hohlca']

Test axefiles per asset and source (4 * 4 * 5 + 4 * 3 * 4 = ~128 tests) , what model to use?
* TCN
* XGBOOST
* RF

OR I just make a best guess on the best features. I record the tensor dimensions (allow easy modification) and get going on the meat of this stuff. I can start with only h/d pba, play with that until it's working well then iteratively add other sources.

I start with one asset, move to others after the first asset's model works.

I know roughly what features to use. I need to now determine the model design.

In [6]:
h_all_common, d_all_common

(['hdmx', 'hdod', 'hdpt', 'hdzn'],
 ['dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd'])

## Select Data

### Choose Features and Join

In [7]:
for k0, v0 in fd.items():
    print(k0)
    for k1,v1 in fd[k0].items():
        print('\t' +k1)
        for k2, v2 in fd[k0][k1].items():
            print('\t\t' +k2)
            for k3, v3 in fd[k0][k1][k2].items():
                print('\t\t\t' +k3)

h
	pba
		hdgau
			pba_hlh_hlogret_hdzn_hdgau(8)_hret_hspread
			pba_hoc_hlogret_hdzn_hdgau(8)_hret_hspread
			pba_hohlca_hdzn_hdgau(8)
		hdmx
			pba_hlh_hlogret_hdmx_hret_hspread
			pba_hoc_hlogret_hdmx_hret_hspread
			pba_hohlca_hdmx
		hdod
			pba_hlh_hlogret_hdod_hret_hspread
			pba_hoc_hlogret_hdod_hret_hspread
			pba_hohlca_hdod
		hdpt
			pba_hlh_hlogret_hdpt_hret_hspread
			pba_hoc_hlogret_hdpt_hret_hspread
			pba_hohlca_hdpt
		hduni
			pba_hlh_hlogret_hdmx_hduni(8)_hret_hspread
			pba_hoc_hlogret_hdmx_hduni(8)_hret_hspread
			pba_hohlca_hdmx_hduni(8)
		hdzn
			pba_hlh_hlogret_hdzn_hret_hspread
			pba_hoc_hlogret_hdzn_hret_hspread
			pba_hohlca_hdzn
		hohlca
			pba_hohlca
	vol
		hdgau
			vol_hlh_hlogret_hdzn_hdgau(8)_hret_hspread
			vol_hoc_hlogret_hdzn_hdgau(8)_hret_hspread
			vol_hohlca_hdzn_hdgau(8)
		hdmx
			vol_hlh_hlogret_hdmx_hret_hspread
			vol_hoc_hlogret_hdmx_hret_hspread
			vol_hohlca_hdmx
		hdod
			vol_hlh_hlogret_hdod_hret_hspread
			vol_hoc_hlogret_hdod_hret_hspread


In [8]:
#for ser_name in chosen_f.index.unique(level='id1'):
#    print('\t' +ser_name)
#    ser_subset_df = chosen_f.xs(ser_name, level=1, drop_level=False)
#    ser_subset_df.index = ser_subset_df.index.remove_unused_levels()
#    display(pd_to_np(ser_subset_df))
#    ser_subset_df.isnan()
#    np.isnan(ser_subset_df.values).any()

In [9]:
from xg_util import get_hardcoded_daily_dfs, get_hardcoded_hourly_dfs

In [10]:
def df_filter_by_keywords(fdf, keywords):
	cols = [col for col in fdf.index.levels[1] if (any(keyword in col for keyword in keywords))]
	return pd_rows_key_in(fdf, 'id1', cols)

def get_hardcoded_daily_dfs(fd, src):
	"""
	Return hardcoded daily freq feature dfs to use in experiments.
	"""
	feature_dfs = []

	#if (src in ('pba', 'vol')):
	#	feature_dfs.append(fd['d'][src]['dlogret']['{src}_dlh_dlogret'.format(src=src)])

	if (src in ('pba', 'vol', 'buzz')):
		#for axe in ('dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd'):
		for axe in ('dwrmx', 'dwrod', 'dwrpt', 'dwrzn'):
			for fdf in fd['d'][src][axe].values():
				#feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low')))
				#feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low', 'close')))
				feature_dfs.append(df_filter_by_keywords(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low')), ('(5)', '(20)',)))

	if (src in ('nonbuzz',)):
		for axe in ('dffd', 'dwrxmx'):
			for fdf in fd['d'][src][axe].values():
				feature_dfs.append(fdf)

	#return pd.concat(feature_dfs).sort_index(axis=0)
	return pd.concat(feature_dfs)

def get_hardcoded_hourly_dfs(fd, src):
	"""
	Return hardcoded hourly freq feature dfs to use in experiments.
	"""
	feature_dfs = []

	if (src in ('pba', 'vol', 'buzz')):
		for axe in ('hdmx', 'hdod', 'hdpt', 'hdzn', 'hdgau', 'hduni'):
			for fdf in fd['h'][src][axe].values():
				feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low', 'close')))

	return pd.concat(feature_dfs)

In [11]:
d_pba = get_hardcoded_daily_dfs(fd, 'pba')
d_vol = get_hardcoded_daily_dfs(fd, 'vol')
d_buzz = get_hardcoded_daily_dfs(fd, 'buzz')
d_nonbuzz = get_hardcoded_daily_dfs(fd, 'buzz')
h_pba = get_hardcoded_hourly_dfs(fd, 'pba')
h_vol = get_hardcoded_hourly_dfs(fd, 'vol')
h_buzz = get_hardcoded_hourly_dfs(fd, 'buzz')

In [12]:
chosen_f = d_pba
chosen_f = fd['d']['pba']['dret']['pba_hlh_hdxret_dret']
chosen_f = fd['d']['pba']['dret']['pba_hoc_hdxret_dret']
#chosen_f = fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir']

In [13]:
# Random 'Features':
rand_f = pd.DataFrame(np.random.rand(len(chosen_f)), index=chosen_f.index).rename({chosen_f.index.levels[1][0]: 'rand'})

In [14]:
chosen_f = rand_f

In [15]:
chosen_f

,,0
id0,id1,
1996-01-02,rand,0.192221
1996-01-03,rand,0.902566
1996-01-04,rand,0.806705
1996-01-05,rand,0.179939
1996-01-08,rand,0.360940
...,...,...
2018-01-02,rand,0.525592
2018-01-03,rand,0.792139
2018-01-04,rand,0.011425


### Choose Labels/Targets and Process

In [16]:
# predict ddir(t)
chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir'].replace(to_replace=-1, value=0))
chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret'])

In [14]:
# predict thresholded direction: ddir1(t)
chosen_l_len = len(ld['hoc']['pba']['ddir1_log'].columns)
chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir1_log'])
chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
chosen_l = (chosen_l[chosen_l!=0] + chosen_l_len).fillna(0).astype(int)
chosen_l = df_add_midx_level(chosen_l, 'pba_hoc_hdxret1_ddir1', loc=1, name='id1')

chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret1_log']).abs().max(level=0)
chosen_t = df_add_midx_level(chosen_t, 'pba_hoc_hdxret1_dret1', loc=1, name='id1')
#chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
#chosen_l

In [15]:
# 'Predict' the present ddir(t-1)
chosen_l = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir'].rename(columns={-1:'pba_hoc_hdxret_ddir'}), loc=1).replace(to_replace=-1, value=0).astype(int))
chosen_t = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['dret']['pba_hoc_hdxret_dret'].rename(columns={-1:'pba_hoc_hdxret_dret'}), loc=1))

### Get Common Indexed Rows (Intersect First Level of MultiIndex)

In [17]:
year_interval = ('2009', '2018')
common_idx = midx_intersect(pd_get_midx_level(chosen_f), pd_get_midx_level(chosen_l), pd_get_midx_level(chosen_t))
common_idx = common_idx[(common_idx > year_interval[0]) & (common_idx < year_interval[1])]
feature_df, label_df, target_df = map(compose(partial(pd_rows, idx=common_idx), df_midx_restack), [chosen_f, chosen_l, chosen_t])
assert(all(feature_df.index.levels[0]==label_df.index.levels[0]))
assert(all(feature_df.index.levels[0]==target_df.index.levels[0]))

In [18]:
feature_df

,,0
id0,id1,
2009-01-02,rand,0.552609
2009-01-05,rand,0.701130
2009-01-06,rand,0.923545
2009-01-07,rand,0.131317
2009-01-08,rand,0.327949
...,...,...
2017-12-22,rand,0.395532
2017-12-26,rand,0.435001
2017-12-27,rand,0.220325


## PL Model

In [19]:
from model_util import init_layer, assert_has_shape_attr

In [20]:
class FFResidualBlock(nn.Module):
	"""
	Wrap a FF residual connection around a network
	"""
	def __init__(self, net, act):
		"""
		Args:
			net (nn.Module): nn.Module to wrap
			act (str): activation function
		"""
		super(FFResidualBlock, self).__init__()
		self.net = net
		self.downsample = init_layer(nn.Linear(net.in_shape[0], net.out_shape[0], bias=True), act=act, init_method='kaiming_normal')
		self.out_act = PYTORCH_ACT_MAPPING.get(act)()

	def forward(self, x):
		residual = x if (isnt(self.downsample)) else self.downsample(x)
		try:
			net_out = self.net(x)
			return self.out_act(net_out + residual)
		except RuntimeError as e:
			print(e)
			logging.error('self.net(x).shape:  {}'.format(self.net(x).shape))
			logging.error('residual.shape: {}'.format(residual.shape))
			sys.exit(0)

class FFNet(nn.Module):
	"""
	"""
	def __init__(self, in_shape, block_sizes=[[20]], block_act='elu', out_act='relu', global_dropout=.2, no_dropout=[0],
                block_init='normal', out_init='normal', out_shapes=[2]):
		"""
		Args:
			in_shape (tuple): shape of the network's input tensor, expects a shape (in_channels, in_width)
			block_sizes (list * list): FF layer sizes in each block, or individual channel sizes per block in sequence
			block_act (str): activation function of each layer in each block
			out_act (str): output activation of each block
			global_dropout (float): dropout probability of an element to be zeroed for any layer not in no_dropout
			no_dropout (list): list of global layer indices to disable dropout on
			block_init (str): 
			out_init (str): 
			out_shapes (list): 
		"""
		super(FFNet, self).__init__()
		self.in_shape = block_in_shape = (in_shape[0]*in_shape[1], )
		blocks = []
		i = 0

		for b in range(len(block_sizes)):
			block_channel_list = block_sizes[b]
			layer_in_shape = block_in_shape
			layers = []

			for l, layer_out_shape in enumerate(block_channel_list):
				layer_out_shape = (layer_out_shape,)
				layers.append(('FF_{b}_{l}'.format(b=b, l=l, i=i), init_layer(nn.Linear(layer_in_shape[0], layer_out_shape[0], bias=True), act=block_act, init_method=block_init)))
				layers.append(('AF_{b}_{l}'.format(b=b, l=l, i=i), PYTORCH_ACT_MAPPING.get(block_act)()))
				if (i not in no_dropout):
					layers.append(('DO_{b}_{l}'.format(b=b, l=l, i=i), nn.Dropout(global_dropout)))
				layer_in_shape = layer_out_shape
				i += 1
			block_out_shape = layer_out_shape
			net = nn.Sequential(OrderedDict(layers))
			net.in_shape, net.out_shape = block_in_shape, block_out_shape
			blocks.append(('RB_{b}'.format(b=b), FFResidualBlock(net, out_act)))
			block_in_shape = block_out_shape
		self.out_shape = block_out_shape
		self.ffnet = nn.Sequential(OrderedDict(blocks))

	def forward(self, x):
		return self.ffnet(x)

In [21]:
class RNN(nn.Module):
	"""
	"""
	def __init__(self, in_shape, block_sizes=[[20]], block_act='elu', out_act='relu', global_dropout=.2, no_dropout=[0],
                block_init='normal', out_init='normal', out_shapes=[2]):
		"""
		Args:
			in_shape (tuple): shape of the network's input tensor, expects a shape (in_channels, in_width)
			block_sizes (list * list): FF layer sizes in each block, or individual channel sizes per block in sequence
			block_act (str): activation function of each layer in each block
			out_act (str): output activation of each block
			global_dropout (float): dropout probability of an element to be zeroed for any layer not in no_dropout
			no_dropout (list): list of global layer indices to disable dropout on
			block_init (str): 
			out_init (str): 
			out_shapes (list): 
		"""
		super(FFNet, self).__init__()
		self.in_shape = block_in_shape = (in_shape[0]*in_shape[1], )
		blocks = []
		i = 0

		for b in range(len(block_sizes)):
			block_channel_list = block_sizes[b]
			layer_in_shape = block_in_shape
			layers = []

			for l, layer_out_shape in enumerate(block_channel_list):
				layer_out_shape = (layer_out_shape,)
				layers.append(('FF_{b}_{l}'.format(b=b, l=l, i=i), init_layer(nn.Linear(layer_in_shape[0], layer_out_shape[0], bias=True), act=block_act, init_method=block_init)))
				layers.append(('AF_{b}_{l}'.format(b=b, l=l, i=i), PYTORCH_ACT_MAPPING.get(block_act)()))
				if (i not in no_dropout):
					layers.append(('DO_{b}_{l}'.format(b=b, l=l, i=i), nn.Dropout(global_dropout)))
				layer_in_shape = layer_out_shape
				i += 1
			block_out_shape = layer_out_shape
			net = nn.Sequential(OrderedDict(layers))
			net.in_shape, net.out_shape = block_in_shape, block_out_shape
			blocks.append(('RB_{b}'.format(b=b), FFResidualBlock(net, out_act)))
			block_in_shape = block_out_shape
		self.out_shape = block_out_shape
		self.ffnet = nn.Sequential(OrderedDict(blocks))

	def forward(self, x):
		return self.ffnet(x)

In [22]:
from model.model_util import TemporalConvNet

In [27]:
m_params = {
	'pad_type': 'same',    
	'num_blocks': 1,
	#'block_channels': [[80, 80, 80]],
	'block_channels': [[70, 70, 70]],
	#'block_channels': [[70]],
	#'block_channels': [[80, 80, 80]],
	#'block_channels': [[80, 70, 60]],
	#'block_channels': [[70, 60, 50]],
	'block_act': 'relu',
	'out_act': 'relu',
	'block_init': 'kaiming_normal',
	'out_init': 'kaiming_normal',
	'kernel_sizes': [7],
	'dilation_index': 'global',
	'global_dropout': 0,
	'no_dropout': [0],
	'out_shapes': [2] # directions * slots per direction
}
t_params = {
	'window_size': 20,
	'flatten_features': False,
	'shuffle_features': False,
	'epochs': 500,#500,
	'batch_size': 64*2,
	'loss': 'ce',
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
#		'name': 'rpl',
#		'kwargs': {
#			'mode': 'min',
#			'factor': 0.1,
#			'patience': 10,
#			'threshold': 0.0001,
#			'threshold_mode': 'rel',
#			'cooldown': 0,
#			'min_lr': 0
#		}
	}
}

In [28]:
mdl = GenericModel(TemporalConvNet, m_params, t_params, (feature_df, label_df, target_df))
display(mdl)

GenericModel(
  (loss): CrossEntropyLoss()
  (model): OutputLinear(
    (emb): TemporalConvNet(
      (convnet): Sequential(
        (RB_0): ResidualBlock(
          (net): Sequential(
            (TL_0_0): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(1, 70, kernel_size=(7,), stride=(1,), padding=(3,))
                (1): ReLU()
                (2): Dropout(p=0, inplace=False)
              )
            )
            (TL_0_1): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(70, 70, kernel_size=(7,), stride=(1,), padding=(6,), dilation=(2,))
                (1): ReLU()
                (2): Dropout(p=0, inplace=False)
              )
            )
            (TL_0_2): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(70, 70, kernel_size=(7,), stride=(1,), padding=(12,), dilation=(4,))
                (1): ReLU()
                (2): Dropout(p=0, inplace=False)
              )
            )

In [70]:
m_params = {
	'block_sizes': [[200, 200, 200, 200, 200, 200, 200, 200, 200, 200]],
	'block_act': 'relu',
	'out_act': 'gelu',
	'global_dropout': 0,
	'no_dropout': [0],
	'block_init': 'normal',
	'out_init': 'normal',
	'out_shapes': [2]#len(chosen_l.columns)*len(chosen_l.index.levels[1])], # directions * slots per direction
}
t_params = {
	'window_size': 20,
	'flatten_features': True,
	'shuffle_features': True,    
	'epochs': 200,#500,
	'batch_size': 64*2,
	'loss': 'ce',
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
#		'name': 'rpl',
#		'kwargs': {
#			'mode': 'min',
#			'factor': 0.1,
#			'patience': 10,
#			'threshold': 0.0001,
#			'threshold_mode': 'rel',
#			'cooldown': 0,
#			'min_lr': 0
#		}
	}
}

In [65]:
mdl = GenericModel(FFNet, m_params, t_params, (feature_df, label_df, target_df))
display(mdl)

GenericModel(
  (loss): CrossEntropyLoss()
  (model): OutputLinear(
    (emb): FFNet(
      (ffnet): Sequential(
        (RB_0): FFResidualBlock(
          (net): Sequential(
            (FF_0_0): Linear(in_features=20, out_features=200, bias=True)
            (AF_0_0): ReLU()
            (FF_0_1): Linear(in_features=200, out_features=200, bias=True)
            (AF_0_1): ReLU()
            (DO_0_1): Dropout(p=0, inplace=False)
            (FF_0_2): Linear(in_features=200, out_features=200, bias=True)
            (AF_0_2): ReLU()
            (DO_0_2): Dropout(p=0, inplace=False)
            (FF_0_3): Linear(in_features=200, out_features=200, bias=True)
            (AF_0_3): ReLU()
            (DO_0_3): Dropout(p=0, inplace=False)
            (FF_0_4): Linear(in_features=200, out_features=200, bias=True)
            (AF_0_4): ReLU()
            (DO_0_4): Dropout(p=0, inplace=False)
            (FF_0_5): Linear(in_features=200, out_features=200, bias=True)
            (AF_0_5): ReLU()
  

In [29]:
escb = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=30, verbose=False, mode='min')
trainer = pl.Trainer(max_epochs=t_params['epochs'], auto_lr_find=False, gpus=1, amp_level='O1', precision=16)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:Using 16bit precision.


In [30]:
trainer.fit(mdl)

/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
INFO:lightning:
   | Name                                      | Type             | Params
---------------------------------------------------------------------------
0  | loss                                      | CrossEntropyLoss | 0     
1  | model                                     | OutputLinear     | 69 K  
2  | model.emb                                 | TemporalConvNet  | 69 K  
3  | model.emb.conv

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


1

In [24]:
for i in range(5):
    mdl = TCNModel(m_params, t_params, (feature_df, label_df, target_df))
    trainer = pl.Trainer(max_nb_epochs=t_params['epochs'], gpus=1, amp_level='O1', use_amp=True)
    trainer.fit(mdl)

NameError: name 'TCNModel' is not defined

## Debug Data (Runtime Transforms)

### Split into Train/Val/Test and Convert to Numpy Tensor

In [32]:
train_np, val_np, test_np = zip(*map(pd_to_np_tvt, (feature_df, label_df, target_df)))
shapes = np.asarray(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np))))
assert all(np.array_equal(a[:, 1:], b[:, 1:]) for a, b in pairwise(shapes)), 'feature, label, target shapes must be identical across splits'
assert all(len(np.unique(mat.T[0, :]))==1 for mat in shapes), 'first dimension (N) must be identical length in each split for all (feature, label, and target) tensors'

### Runtime Preproc

In [33]:
params = {
    'loss': 'ce',
    'batch_size': 1,
    'window_size': 20,
}

In [34]:
def __preproc__(data, m_params, overlap=True):
    x, y, z = temporal_preproc_3d(data, window_size=m_params['window_size'], apply_idx=[0]) if (overlap) else stride_preproc_3d(data, window_size=m_params['window_size'])
    if (m_params['loss'] in ('bce', 'bcel', 'ce', 'nll')):
        y_new = np.sum(y, axis=(1, 2), keepdims=False)		# Sum label matrices to scalar values
        if (y.shape[1] > 1):
            y_new += y.shape[1]								# Shift to range [0, C-1]
        if (m_params['loss'] in ('bce', 'bcel')):
            y_new = np.expand_dims(y_new, axis=-1)
        y = y_new
    return (x, y, z)

#@pl.data_loader
def train_dataloader(t_params, flt):
    logging.info('train_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def val_dataloader(t_params, flt):
    logging.info('val_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

#@pl.data_loader
def test_dataloader(t_params, flt):
    logging.info('test_dataloader called')
    return batchify(t_params, __preproc__(flt), False)

### Overlapping Episodes:

In [35]:
train_ol_np = __preproc__(train_np, params)
val_ol_np = __preproc__(val_np, params)
test_ol_np = __preproc__(test_np, params)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_ol_np)))
print(np_value_counts(train_ol_np[1]))

((1359, 4, 1), (1359, 1, 2), (1359, 1, 2))
((1340, 4, 20), (1340,), (1340, 1, 2))
(array([0., 1.]), array([593, 747]))


### Non-Overlapping Episodes:

In [612]:
train_nol_np = __preproc__(train_np, params, False)
val_nol_np = __preproc__(val_np, params, False)
test_nol_np = __preproc__(test_np, params, False)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_nol_np)))

((1359, 3, 8), (1359, 1, 2), (1359, 1, 2))
((67, 3, 20, 8), (67, 2), (67, 1, 20, 2))


In [613]:
def batchify(params, data, shuffle_batches=False):
	"""
	Return a torch.DataLoader made from a tuple of numpy arrays.

	Args:
		params (dict): model parameters dictionary
		data (tuple): tuple of numpy arrays, features are the first element
		shuffle_batches (bool): whether or not to shuffle the batches

	Returns:
		torch.DataLoader
	"""
	f = torch.tensor(data[0], dtype=torch.float32, requires_grad=True)
	if (params['loss'] in ('bce', 'bcel', 'mae', 'mse')):
		l = [torch.tensor(d, dtype=torch.float32, requires_grad=False) for d in data[1:]]
	elif (params['loss'] in ('ce', 'nll')):
		l = [torch.tensor(d, dtype=torch.int64, requires_grad=False).squeeze() for d in data[1:]]
	ds = TensorDataset(f, *l)
	dl = DataLoader(ds, batch_size=params['batch_size'], shuffle=shuffle_batches)
	return ds

## Sklearn Tests

In [614]:
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier

In [615]:
x_train, y_train, z_train = train_ol_np
x_val, y_val, z_val = val_ol_np
x_test, y_test, z_test = test_ol_np

In [616]:
def np_collapse_last_two_dim(arr):
    return arr.reshape(arr.shape[0], arr.shape[1]*arr.shape[2])

In [617]:
x_train.reshape(1340, 800)

ValueError: cannot reshape array of size 643200 into shape (1340,800)

In [618]:
x_train_new = np_collapse_last_two_dim(x_train)
x_val_new = np_collapse_last_two_dim(x_val)
x_test_new = np_collapse_last_two_dim(x_test)

In [619]:
# Logistic Regression
clf = LogisticRegression(C=10**-2, l1_ratio=.9, penalty='elasticnet', random_state=0, solver='saga').fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5092165898617511
diff : 0.000000%


In [620]:
# SGDClassifier
clf = SGDClassifier(alpha=10**-2, l1_ratio=.8, loss='log', penalty='elasticnet', random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [621]:
# SGDClassifier
clf = SGDClassifier(alpha=10**-5, l1_ratio=.1, loss='modified_huber', max_iter=1000, penalty='elasticnet', random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5230414746543779
diff : 1.382488%


In [622]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='gini', bootstrap=True, max_depth=4, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [635]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy', bootstrap=True, max_depth=None,
                             min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4769585253456221
diff : -3.225806%


In [624]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=7, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=True, max_depth=3, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4930875576036866
diff : -1.612903%


In [625]:
# RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=True, max_depth=4, min_impurity_decrease=0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.49078341013824883
diff : -1.843318%


In [626]:
# GradientBoostingClassifier
clf = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0,
                                 criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4861751152073733
diff : -2.304147%


In [627]:
# GradientBoostingClassifier
clf = GradientBoostingClassifier(loss='exponential', learning_rate=0.1, n_estimators=100, subsample=1.0,
                                 criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1,
                                 min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.4861751152073733
diff : -2.304147%


In [628]:
# HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(loss='auto', learning_rate=0.1, max_iter=100, max_leaf_nodes=31,
                                     max_depth=4, min_samples_leaf=20, l2_regularization=0.0, max_bins=255, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.46543778801843316
diff : -4.377880%


In [629]:
# RF Alt 1
clf1 = DecisionTreeClassifier(criterion='entropy', max_depth=3, max_features='auto', class_weight='balanced')
clf = BaggingClassifier(base_estimator=clf1, bootstrap=True, bootstrap_features=False, n_estimators=100, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.532258064516129
diff : 2.304147%


In [630]:
# RF Alt 2
clf = RandomForestClassifier(n_estimators=1, class_weight='balanced_subsample', criterion='entropy',
                             bootstrap=False, max_depth=3, min_impurity_decrease=0, random_state=0)
clf = BaggingClassifier(base_estimator=clf1, bootstrap=True, bootstrap_features=False, n_estimators=1000, random_state=0).fit(x_train_new, y_train)
sc = clf.score(x_val_new, y_val)
print('base :', y_val.mean())
print('score:', sc)
print('diff : {:%}'.format((sc-y_val.mean())))

base : 0.5092165898617511
score: 0.5069124423963134
diff : -0.230415%


In [598]:
random_states = range(10)
results = np.zeros(len(random_states))
y_mean = y_val.mean()
for r in random_states:
        # RandomForestClassifier
    clf = RandomForestClassifier(n_estimators=100, class_weight=None, criterion='entropy',
                                 bootstrap=True, max_depth=None, min_impurity_decrease=0, random_state=r).fit(x_train_new, y_train)
    sc = clf.score(x_val_new, y_val)
    diff = sc-y_mean
    results[r] = diff*100
    #print('base :', y_val.mean())
    #print('score:', sc)
    #print('diff : {:%}'.format((sc-y_val.mean())))
print(results)
print(results.mean())

[ 2.76497696 -2.99539171 -3.4562212   2.07373272  1.38248848 -1.61290323
 -0.92165899  0.46082949 -1.84331797 -3.68663594]
-0.783410138248844


In [67]:
# Few Shot Tests
# XXX - Need to update for code changes
tr_split = .75
w = params['window_size']
tr, ts = int(tr_split*w), (1-tr_split)*w
print(tr, ts)

scs = []
for x, y, z in zip(*train_nol_np):
    x, y = x.T, y
    print(x.shape)
    print(x[0:tr].shape)
    print(x[tr:].shape)
    break
    clf = LogisticRegression(C=10**-2, l1_ratio=.9, penalty='elasticnet', random_state=0, solver='saga').fit(x[0:tr], y[0:tr].squeeze())
    #clf = LinearRegression().fit(x[0:tr], y[0:tr].squeeze())
    #yh = clf.predict(x[tr:])
    #|yp = clf.predict_proba(x[tr:])
    sc = clf.score(x[tr:], y[tr:].squeeze())
    scs.append(sc)
    #print(sc)
    #print(x)
    #print(y.squeeze())
    #print(z.T)

15 5.0
(160, 5)
(15, 5)
(145, 5)


In [68]:
(np.array(scs).mean()-train_nol[1].squeeze().mean())*100

/home/kev/crunch/model/nb-model_xg-model-data-fi.ipynb:1: RuntimeWarning: Mean of empty slice.
  {
/home/kev/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'train_nol' is not defined